In [ ]:
#BUILDING A MARKETTING BROUCHRE AI AGENT

In [ ]:
import requests
import ollama
import json #maybe useful when requiring JSON format 
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display


In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

In [ ]:
link_system_prompt = """
You are provided with a list of links found on a webpage. You must decide which of the links are most relevant to include in a brochure about the company, such as About pages, Company pages, Careers/Jobs pages, or other significant links.

Respond **only** in JSON format, with no additional explanation, as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""


In [ ]:
print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(ed))

In [ ]:
def messages_for_links(website):
    return [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(website)}
    ]

In [ ]:
link_system_prompt

In [ ]:
get_links_user_prompt(ed)

In [ ]:
def messages_for_links(website):
    return [
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(website)}
    ]

In [ ]:
messages_for_links(ed)

In [ ]:
def get_links(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for_links(website),
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    api_response = response.json()
    try:
        result = api_response["message"]["content"]
        return json.loads(result)  # Parse result as JSON
    except json.JSONDecodeError:
        return {"error": "API did not return valid JSON", "content": result}

In [ ]:
#print(get_links("https://www.edwarddonner.com"))

In [ ]:
huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://www.nebula.io"))

In [ ]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("NEBULA.io", "https://www.nebula.io")

In [ ]:
def messages_for_brochure(company_name, url):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
    ]

In [ ]:
def create_brochure(company_name, url):
     payload = {
        "model": MODEL,
        "messages": messages_for_brochure(company_name, url ),
        "stream": False
    }
     response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
     return (response.json()['message']['content'])

In [ ]:
create_brochure("NEBULA", "https://www.nebula.io")

In [ ]:
import requests
import json
from IPython.display import Markdown, display

def stream_brochure(company_name, url):
    payload = {
        "model": MODEL,
        "messages": messages_for_brochure(company_name, url),
        "stream": True  # Enable streaming
    }

    # Initialize the display for live updates
    display_handle = display(Markdown(""), display_id=True)
    response_accumulated = ""  # Accumulated response

    try:
        with requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True) as response:
            response.raise_for_status()  # Ensure the request was successful
            
            # Iterate through streamed response lines
            for line in response.iter_lines():
                if line:  # Ignore keep-alive newlines
                    try:
                        data = json.loads(line)
                        if "message" in data and "content" in data["message"]:
                            content = data["message"]["content"]
                            response_accumulated += content
                            # Clean and update the markdown display
                            formatted_response = response_accumulated.replace("```", "").replace("markdown", "")
                            display_handle.update(Markdown(formatted_response))  # Update the display
                    except json.JSONDecodeError as e:
                        print(f"Error decoding JSON: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error with the request: {e}")


In [186]:
stream_brochure("NEBULA", "https://www.nebula.io")

**Welcome to Nebula: The Intelligent Talent Acquisition Platform That's Changing the Game**

[Image of a starry night sky with a bright nebula in the center]

Are you tired of sifting through countless resumes and job boards, only to find that they're not quite what you're looking for? Look no further than Nebula, the intelligent talent acquisition platform that reimagines recruitment.

**Our Mission**

At Nebula, we believe that finding the right person for the job should be easier than ever. That's why we've created a platform that uses cutting-edge technology and machine learning algorithms to connect you with the world's best talent.

**What Sets Us Apart**

* **Access to the World's Best Talent**: Nebula contains the world's best talent, validated by top sources, educators, and employers.
* **Proprietary Algorithms**: Our matching engine analyzes hundreds of millions of profiles in seconds to provide you with the most meaningful matches.
* **Diversity, Equity & Inclusion**: We're committed to fair and equal employment initiatives, ensuring that bias is eliminated from our AI models.

**Our Team**

Meet the team behind Nebula:

* **Steven Talbot**: Co-Founder and CEO
* **Ed Donner**: Co-Founder and CTO
* **Jon Krohn**: Co-Founder and Chief Data Scientist
* **Alex Gardner**: CFO
* **Mike Evers**: Strategy & GTM

**What Our Customers Say**

"Don Nebula has revolutionized our recruitment process. We've saved time, reduced bias, and found top talent faster than ever before." - Fortune 500 Client

**Ready to Experience the Future of Recruitment?**

Schedule a demo today and see how Nebula can transform your hiring process.

**Try It For Free**

Take Nebula for a spin and discover why we're recognized as one of the most innovative companies in recruitment.

[Image of a person working on a laptop with a satisfied expression]

**Join Our Mission**

Be part of a team that's changing the face of talent acquisition. Explore our careers page to find out how you can join the Nebula crew!

[Link to Careers Page]